# A Beginner's Guide to Perform Molecular Dynamics Simulation of Liquid-Liquid Interfaces using GROMACS

Instructions for running a GROMACS simulation of a biphase and a triphase liquid system. Based on the GROMACS tutorials on biphasic systems: https://tutorials.gromacs.org/tutorial-biphase-new.html, http://www.mdtutorials.com/gmx/biphasic/index.html. These resources are intended for pedagogical purposes, and were designed for the undergraduate and third-cycle courses at Politecnico di Torino. 

Michele Pellegrino (michele.pellegrino@polito.it)

All rights reserved (2025)

## Motivation

1. Computing the **surface tension** of a liquid-liquid interface is a very common procedure in flow and wetting simulations, and in multiscale modelling. Nonetheless, it hides a few technicalities that is worth discussing.
2. **Surfactant contamination** is a very serious problem in many industrial processes and technologies, some enegy-related (e.g. liquid infused surfaces for drag reduction). Even a small concentration of surfactants can have a dramatic effect on surface tension. 

**[!FIGURE OF LIS!]**

We are going to simulate the formation of an interface between immiscible and partially-miscible liquids. The three liquids we will be considering are **water**, **hexane** and **pentanol**. The first two do not mix with each other at room conditions, while the latter is partially soluble in water. Pentanol is added to the system in order to observe how adsorption of alchol molecules at an interface may affect surface tension.

## Introduction

Let's first import all the modules needed for this exercise:

In [ ]:
import nglview as ng
import MDAnalysis as mda

import pandas as pd
import os

Let's also store the work directory, to avoid using relative paths:

In [ ]:
workdir = os.getcwd()
print("Work directory:",workdir)

As you already know, in Molecular Dynamics each molecule has to be parameterized according to a force field. For water, we will be using the **SPC/E** model (https://doi.org/10.1021/j100308a038). The parameters for hexane and pentanol are taken from the **OPLS-AA** force field, i.e the all-atoms Optimized Potentials for Liquid Simulations (https://doi.org/10.1021/ja9621760).

Let's explore the input configurations:

In [ ]:
!tar -xvf data.tar.gz

In [ ]:
%cd {workdir}/data
!ls hexane

The `data` folder already contains the topology (`1-hex.itp`) and the structure (`1-hex.gro`) of one hexane molecule. Topologies for alkanes and alcohols can be found in the literature. Another option is to use a server like LigParGen (https://zarbi.chem.yale.edu/ligpargen/), although the results need to be taken with a grain of salt. In our case we used the parameters for the methyl groups in in `alkanes.rtp`, which can be found in `gromacs/share/gromacs/top/oplsaa.ff`.

In [ ]:
!cat hexane/1-hex.itp

In [ ]:
view_hex = ng.show_structure_file("hexane/1-hex.gro")
view_hex

## Preparing the biphasic system

The first step in this exercise will be to fill an empty box with hexane and water. There exist (at least) three GROMACS commands that can be used to generate a liquid box: `gmx insert-molecules`, `gmx solvate` and `gmx genconf`. When starting with a single molecule, `gmx insert-molecules` is preferable as it indirectly allows to target a desired density.

We know that the density of liquid hexane at room condition is roughly 660 kg/m^3, while its molecular mass is 86.178 u. We guess that in order to fill a 3x3x3 nm^3 box we need roughly 125 molecules. When `gmx insert-molecules` is called, GROMACS tries to fit the requested number of molecules in a box, but it might not be able to do it in its first try. We can specify the number of insertion tries per molecule using the `-try` flag; the higher, the closer the number of inserted molecules will be to the specified one. Mind that 'forcing' a lot of molecules in a large box might be extremely slow and lead to systems that are difficult to equilibrate.

**Tip**: 1 u/nm^3 ≈ 1.660 kg/m^3

<span style="color:red"> **Exercise** - complete the call to `gmx insert-molecules` by adding the flags for the **size of the simulation box** and the **number of molecules** (and the relative input). You can have a look at the documentation or inspect `gmx help insert-molecules`.</span>

In [ ]:
# TODO: remove the flags in the exercise
!gmx insert-molecules -ci hexane/1-hex.gro -box 3 3 3 -nmol 125 -try 100 -seed 1234 -o hexane-box.gro

Let's have a look a the the result:

In [ ]:
view_hex_box = ng.show_structure_file("hexane-box.gro")
view_hex_box

Possibly, the number of molecules will not match the one specified with `-nmol`. It doesn't really matter since there's no guarantee that the experimental density will match the one of simulated fluids anyway. The equilibrium density will be obtained after running the equilibration simulation at constant pressure.

The code snippet below reads the number of atoms from the generated `.gro` file and append the number of hexane molecules (residues) to the topology file. **If you intend to run this tutorial multiple times, remember to delete the last two lines of `topology-biphase.top` and the last three lines of `topology-triphase.top`**.

In [ ]:
with open("hexane-box.gro") as fp:
    for i, line in enumerate(fp):
        if i == 1 :
            nmol = int(line)//20
            break
with open("topology-biphase.top", "a") as fp :
    fp.write("HEX "+str(nmol)+"\n")
with open("topology-triphase.top", "a") as fp :
    fp.write("HEX "+str(nmol)+"\n")

In [ ]:
!cat topology-biphase.top

Now that we have a (non-equilibrated!) box of hexane, we can start building the biphasic system by increasing the size of the simulation box and adding water.

The easier way to create the biphasic system is to increase the size of the simulation box and 'solvate' it with water. The end result will be a system that is homogeneous along x and y, and roughly half-filled by water and half-filled by hexane along z. Due to the use of periodic boundary conditions, two water-hexane interfaces will be created. As a matter of personal taste, I find having one interface located close to the periodic edge very *unsettling*. Hence, we are also going to center the system on the hexane phase. Mind that this is purely optional and it may be practically better in some situations to have hexane or water occupying the lower or upper half instead.

In [ ]:
!gmx editconf -box 3.0 3.0 6.0 -c -f hexane-box.gro -o hexane-vacuum.gro

Unfortunately, it is not possible to visualize the periodic box cell with `nglview`. Using VMD one should see something like this:

**[!FIGURE!]**

In [ ]:
# If you have VMD, install the periodic box plugin to visualize the periodic box
!vmd hexane-vacuum.gro

We will now 'fill' the empty space with water using `gmx solvate`. Let's have a look at the documentation:

In [ ]:
!gmx help solvate

The file `water-box.gro` contains the coordinates of a water box that has already been equilibrated. This will be the input to `-cs`. In order to avoid too many water molecules 'spawning' inside the hexane phase, we might have to tune the `-scale` parameter of `gmx solvate`. Furthermore, we are appending the number of instered water molecules to `topology-biphase.top`. 

<span style="color:red"> **Exercise** - based on the documentation of `gmx solvate`, determine the input file to be passed via the `-cp` flag.</span>

In [ ]:
# TODO: remove the input to the -cp flag
!gmx solvate -cp hexane-vacuum.gro -cs water-box.gro -o biphase.gro -scale 0.65 -p topology-biphase.top

In [ ]:
!cat topology-biphase.top

In [ ]:
view_wathex = ng.show_structure_file("biphase.gro")
# Again, this line is necessary to visualize water. 
# You may change the representation type to anything you like!
view_wathex.add_representation('licorice',selection="water")
view_wathex

Some water molecules may ended up in the section of the domain occupied by hexane. They will diffuse into the water phase during the equilibration simulation. There are smarter ways to prepare a biphasic system to ease equilibration, especially if the system is very large. For the purposes of this tutorial the approach detailed above will suffice.

## Simulating the biphasic system

Now we mimick the workflow of the previous exercises: energy minimization via steepest descent and then equilibriation. The novely is that now we will also perform an equilibration step at constant _pressure_ (i.e. $NPT$ ensemble).

In [ ]:
%cd {workdir}
!tar -xvf runs.tar.gz

You can have a look at the `.mdp` files in `runs/EM-biphase/`, `runs/NVT-biphase/` and `runs/NPT-biphase/`.

### Energy minimization

In [ ]:
%cd {workdir}/runs

In [ ]:
"""
    .mdp file for the steepest-descent algorithm,
    which is a way to perform energy minimization
"""
!cat EM-biphase/sd.mdp

In [ ]:
# Preparing the energy minimization simulation
!gmx grompp -p ../data/topology-biphase.top -f EM-biphase/sd.mdp -c ../data/biphase.gro -o EM-biphase/system.tpr

In [ ]:
# Running energy minimization
%cd {workdir}/runs/EM-biphase
!gmx mdrun -v -s system.tpr

### Equilibration at constant volume (NVT)

In [ ]:
%cd {workdir}/runs

As for the CNT and the zeolite exercises, we are going to use a _stochastic velocity rescale_ thermostat to set and control the temperature of the system:

$$dK = (K_0-K)\frac{dt}{\tau_T} + 2\sqrt{\frac{KK_0}{N_f\tau_T}}dW,$$

being $K$ total the kinetic energy, $K_0$ the imposed kinetic energy (aka imposed _temperature_), $\tau_T$ the relaxation time and $N_f$ the number of degrees of freedom in the system. 

This is a **Langevin equation**, that GROMACS solves concurrently with the simulation. The first term represents a linear first-order relaxation, with $dt$ being the differential in time. The second term represents a fluctuating force and $dW$ is the differential on a Brownian process (its definition is far from trivial!).

<span style="color:red"> **Exercise** - in `NVT-biphase/nvt.mdp`, specify the thermostat type (stochastic velocity rescale), the coupling group (the whole system), the relaxation time (1ps) and the reference temperature (300K). You can find the documentation of `.mdp` options here: https://manual.gromacs.org/current/user-guide/mdp-options.html</span>

In [ ]:
# TODO
"""
    .mdp file for NVT equilibration
    Note that the time-step is 1fs. This value has been chosen for numerical stability,
    since the initial configuration is not equilibrated.
"""
!cat NVT-biphase/nvt.mdp

In [ ]:
# Preparing a short NVT equilibration simulation
!gmx grompp -p ../data/topology-biphase.top -f NVT-biphase/nvt.mdp -c EM-biphase/confout.gro -o NVT-biphase/system.tpr

In [ ]:
# Running NVT equilibration
%cd {workdir}/runs/NVT-biphase
!gmx mdrun -v -s system.tpr

Let's make sure that temperature has converged (you can open `.xvg` files with `xmgrace` instead).

In [ ]:
!echo 'Temperature' | gmx energy -f ener.edr -o temperature.xvg -xvg none

df = pd.read_csv('temperature.xvg', sep='\s+', header=None, names=['Time (ps)','T (K)'])
df.plot('Time (ps)')

### Equilibration at constant pressure (NPT) and production

In [ ]:
%cd {workdir}/runs

In [ ]:
"""
    .mdp file for NPT simulation.
    Note that now the time-step is 2fs, and that we are using a baarostat (C-rescale)
"""
!cat NPT-biphase/npt.mdp

In [ ]:
# Preparing the NPT simulation
!gmx grompp -p ../data/topology-biphase.top -f NPT-biphase/npt.mdp -c NVT-biphase/confout.gro -o NPT-biphase/system.tpr

In [ ]:
# Running the NPT simulation
# You can reduce the number of simulation steps using the -nsteps flag, at your own risk!
%cd {workdir}/runs/NPT-biphase
!gmx mdrun -v -s system.tpr

### Pressure control

While the simulation is running, let's spend some words on pressure control. An $NPT$ ensemble is obtained by combining a _thermostat_ and a _barostat_. We are using a _stochastic cell rescale_ barostat:

$$d\epsilon = -\frac{\beta}{\tau_P}(P_0-P)dt + \sqrt{\frac{2k_BT\beta}{V\tau_P}}dW,$$

where $\epsilon$ is the simulation cell _strain_, $\beta$ is the system's compressibility, $P$ is the system's pressure, $P_0$ is the desired pressure, $k_B$ is the Boltzmann's constant, $T$ is the system's temperature and $V$ is the system't volume.

This Langevin equation is formally equivalent to the unsed for temperature control, with a few catches:
- The controlled variable is not pressure itself (or a proxy), but the _strain_ (hence stochastic _cell_ rescale);
- We have an additional 'compressibility parameter'.

Moreover: how the heck is pressure (on general stresses) defined in molecular dynamics?

**Strains vs. stresses**

While the relation between $T$ and $K$ is determined by statistical mechanics (i.e. equipartition theorem), the same doesn't hold for the relation between $P$ and $\epsilon$, which in general results from a costitutive relation. GROMACS offers different types of coupling types (`pcoupltype`), which should be used in different contexts:
- `pcoupltype = isotropic` : all cell strains are scaled equally, good for liquids;
- `pcoupltype = semiisotropic` : $x/y$ strains are scaled separately from $z$ strains, good for membranes;
- `pcoupltype = anisotropic` : each strain is scaled separately, good for solids (especially crystals).

**[!Figures of examples of each case (liquid box, membrane, solid crystal)!]**

**Compressibility**

This parameter always causes a lot of confusion to people approaching Molecular Dynamics simulations. If you look the equation for pressure control, $\beta$ and $\tau_P$ always come together. Hence, reducing the system's compressibility is tantamount as increasing the relaxation time (slower equilibration), and vice-versa. Hence, even if $\beta$ is a bit off, there should be no problem when running equilibrium simulations of monophasic systems or systems of similar composition (e.g. liquid+liquid or liquid+membrane). 

Things are a bit more complicated for non-equilibrium simulations and systems of multiple phase with _substantially_ different pressure response (e.g. liquid+vapour) .

**Virial**

If you recall continous mechanics, stresses are _surface forces_ (as opposed to _volume forces_, like gravity, electric fields, ...). But we don't have any _surface_ in Molecular Dynamics, just a bunch of point masses and charges. How is pressure defined then?

The pressure tensor can be defined in the following way (see a good StatMec book for a derivation, like the one by Tuckerman):

$$\boldsymbol{P} = \frac{2}{V}(\boldsymbol{E}_{kin}-\boldsymbol{\Xi}) ,$$

with:

$$\boldsymbol{E}_{kin}=\frac{1}{2}\sum_im_i\boldsymbol{v}_i\otimes\boldsymbol{v}_i , \quad \boldsymbol{\Xi}=-\frac{1}{2}\sum_{i<j}\boldsymbol{r}_{ij}\otimes\boldsymbol{F}_{ij}.$$

The first term is the kinetic energy tensor (think of the law of perfect gas: $PV=nRT$). The second term is the _virial stess_, and it encapsulates the measure of the response of the system to deformations. Try to imagine how $\boldsymbol{\Xi}$ changes as the system is dilated or compressed, in the case of a solid crystal (e.g. assuming simple Lennard-Jones interactions).

As usual, let's visualize the trajectory:

In [ ]:
u_wh = mda.Universe('system.tpr', 'traj_comp.xtc')
view_wh = ng.show_mdanalysis(u_wh)
view_wh.clear_representations()
view_wh.add_representation('point',pointSize=2,selection="water")
view_wh.add_representation('point',pointSize=2,selection="HEX")
view_wh

In [ ]:
# If you have VMD installed
!vmd traj_comp.xtc confout.gro

Let's check if the box volume has stabilized. Since we are running with `pcoupltype = isotropic`, it is sufficient to inspect one of the three box edge lengths.

In [ ]:
!echo 'Box-Z' | gmx energy -f ener.edr -o box-z.xvg -xvg none

In [ ]:
# If you have xmgrace installed
# It won't show any label on the axes due to the "-xvg none" flag we passed to "gmx energy"
!xmgrace -free box-z.xvg

In [ ]:
# Plot using pandas
df = pd.read_csv('box-z.xvg', sep='\s+', header=None, names=['Time (ps)','Box z (nm)'])
df.plot('Time (ps)')

## Extracting density profiles and surface tension

### Density profiles

GROMACS command `gmx density` can be used to obtain the density profile along one of the 3 cartesian axis of the simulation box. Let's have a look at how it is used:

In [ ]:
!gmx help density

We are going to compute the density profile of hexane and water separately, and compare the two. The total density profile can be computed instead. Note that we are using only the second half of the simulation (`-b` flag), as we want to make sure the system is at equilibrium.

In [ ]:
!echo 'HEX' | gmx density -f traj_comp.xtc -s system.tpr -b 250 -o density-profile-hex.xvg -xvg none
!echo 'SOL' | gmx density -f traj_comp.xtc -s system.tpr -b 250 -o density-profile-sol.xvg -xvg none

In [ ]:
df_hex = pd.read_csv('density-profile-hex.xvg', sep='\s+', header=None, names=['z (nm)','density hexane (kg/m^3)'])
df_sol = pd.read_csv('density-profile-sol.xvg', sep='\s+', header=None, names=['z (nm)','density water (kg/m^3)'])
df = df_hex.merge(df_sol)
df.plot('z (nm)')

The density profiles over $z$ indicate that the two phases are completely separated. The locations of the water/hexane interfaces correspond to the regions of sharp density gradients. Usually, the positions of the interfaces are defined as the points where the density of water and the density of hexane are equal, which are easy to identify from the plot above (simply look where the two curves intersect).

### Surface tension

Next, we extract the surface tension from `ener.edr`. The way `gmx energy` computes the surface tension is quite simple, as it is essentially extracted from the difference between the diagonal component of the pressure tensor that is parallel to the interface and the average of the other two components:

$$
    \gamma = \frac{L_z}{n}\left\{P_{zz}-\frac{P_{xx}+P_{yy}}{2}\right\} \; ,
$$

being $n$ the number of interfaces. This is usually referred to as the _Kirkwood-Buff_ formula. 

Conventionally, the interface is always parallel to the $z$ coordinate. Since Gromacs *doesn't know* a priori how many interfaces does the system have, the output of `gmx energy` is actually $n\cdot\gamma$. Therefore, we will have to divide that number by 2.

In [ ]:
!echo '#Surf*SurfTen' | gmx energy -f ener.edr -s system.tpr -b 250 -o surf-tens-x2.xvg

The number I got is roughly 581.38 bar$\times$nm = 58.138 dyn/cm, which is in very good agreement with experimental values (even if SPC/E is not the most accurate water model!).

## Introducing pentanol

In the last part of this exercise we will introduce a third molecular species, and observe how the now triphase system behaves. The liquid of choice is pentanol, which is an *alcohol*. Given that alcohol molecules have a hydrophilic hydroxyl group and an hydrophobic methyl chain, we can make the 'educated' guess that pentanol would act as a **surfactant**, accumulating at the interface between water and hexane.

In [ ]:
%cd {workdir}/data/

In [ ]:
!cat pentanol/1-pentanol.itp

In [ ]:
view_pen = ng.show_structure_file("pentanol/1-pentanol.gro")
view_pen

We insert some pentanol molecules into the hexane/vacuum configuration, which will be then solvated with water. I figured out that some *interesting* behavior can be observed with around 28 pentanol molecules.

In [ ]:
!gmx insert-molecules -ci pentanol/1-pentanol.gro -f hexane-vacuum.gro -nmol 28 -try 100 -seed 4242 -o hexane-pentanol.gro

In [ ]:
with open("hexane-vacuum.gro") as fp:
    for i, line in enumerate(fp):
        if i == 1 :
            nhex = int(line)//20
            break
with open("hexane-pentanol.gro") as fp:
    for i, line in enumerate(fp):
        if i == 1 :
            npen = (int(line)-20*nhex)//18
            break
with open("topology-triphase.top", "a") as fp :
    fp.write("PEN "+str(npen)+"\n")

Let's check if the topology file looks correct, we expect to find 28 pentanol molecules:

In [ ]:
!cat topology-triphase.top

In [ ]:
view_hexpen = ng.show_structure_file("hexane-pentanol.gro")
view_hexpen

Let's not fill the remaining empty space with water, using `solvate` as for the water-hexane system:

In [ ]:
!gmx solvate -cp hexane-pentanol.gro -cs water-box.gro -o triphase.gro -scale 0.65 -p topology-triphase.top

In [ ]:
view_triphase = ng.show_structure_file("triphase.gro")
view_triphase.add_representation('licorice',selection="water")
view_triphase

## Simulating the adsorption of pentanol to the water/hexane interface

Now that we have 'sprinkled' some pentanol molecules into the simulation box, we are going to run the usual equilibration runs. As the system tries to reach its free-energy minimum, we should observe adsorption of most or all pentanol molecules to the water/hexane interface. Otherwise, the workflow is very similar to the one for the biphasic system.

### Energy minimization

In [ ]:
%cd {workdir}/runs/

In [ ]:
!cat EM-triphase/sd.mdp

In [ ]:
!gmx grompp -p ../data/topology-triphase.top -f EM-triphase/sd.mdp -c ../data/triphase.gro -o EM-triphase/system.tpr

In [ ]:
%cd {workdir}/runs/EM-triphase
!gmx mdrun -v -s system.tpr

### NVT equilibration

In [ ]:
%cd {workdir}/runs/

In [ ]:
!cat NVT-triphase/nvt.mdp

In [ ]:
!gmx grompp -p ../data/topology-triphase.top -f NVT-triphase/nvt.mdp -c EM-triphase/confout.gro -o NVT-triphase/system.tpr

In [ ]:
%cd {workdir}/runs/NVT-triphase
!gmx mdrun -v -s system.tpr

In [ ]:
!echo 'Temperature' | gmx energy -f ener.edr -o temperature.xvg -xvg none

df = pd.read_csv('temperature.xvg', sep='\s+', header=None, names=['Time (ps)','T (K)'])
df.plot('Time (ps)')

### NPT equilibration and production

In [ ]:
%cd {workdir}/runs/

In [ ]:
!cat NPT-triphase/npt.mdp

In [ ]:
!gmx grompp -p ../data/topology-triphase.top -f NPT-triphase/npt.mdp -c NVT-triphase/confout.gro -o NPT-triphase/system.tpr

In [ ]:
%cd {workdir}/runs/NPT-triphase
!gmx mdrun -v -s system.tpr

In [ ]:
!echo 'Box-Z' | gmx energy -f ener.edr -o box-z.xvg -xvg none

df = pd.read_csv('box-z.xvg', sep='\s+', header=None, names=['Time (ps)','Box z (nm)'])
df.plot('Time (ps)')

The adsorption of pentanol to the interface can be verified by visual inspection. I am again using the surface representation in `nglviewer` to highlight the location where pentanol molecules are adsorbing (light blue clouds):

In [ ]:
u_whp = mda.Universe('system.tpr', 'traj_comp.xtc')
view_whp = ng.show_mdanalysis(u_whp)
view_whp.clear_representations()
view_whp.add_representation('surface', 
    selection="PEN", probeRadius=0.6, colorValue = 'yellow')
view_whp.add_representation('point',pointSize=2,selection="water")
view_whp.add_representation('point',pointSize=2,selection="HEX")
view_whp

### Measuring the effect of pentanol adsorption on the density profile and the surface tension

In [ ]:
!echo 'PEN' | gmx density -f traj.trr -s system.tpr -b 500 -o density-profile-pentanol.xvg -xvg none
!echo 'HEX' | gmx density -f traj.trr -s system.tpr -b 500 -o density-profile-hexane.xvg -xvg none
!echo 'SOL' | gmx density -f traj.trr -s system.tpr -b 500 -o density-profile-water.xvg -xvg none

In [ ]:
df_pen = pd.read_csv('density-profile-pentanol.xvg', sep='\s+', header=None, names=['z (nm)','density pentanol (kg/m^3)'])
df_hex = pd.read_csv('density-profile-hexane.xvg', sep='\s+', header=None, names=['z (nm)','density hexane (kg/m^3)'])
df_wat = pd.read_csv('density-profile-water.xvg', sep='\s+', header=None, names=['z (nm)','density water (kg/m^3)'])
df = df_pen.merge(df_hex)
df = df.merge(df_wat)
df.plot('z (nm)')

The diffused 'spikes' of pentanol density close to the water/hexane interface are a signature of adsorption.

Lastly, we compute the surface tension. Usually, surfactant molecules have the effect of reducing the interfacial tension between two liquids. Other than being a very common occurrence (e.g. think about the effects of adding soap to water), surfactant addition or pollution have very important biological and industrial implications.

In [ ]:
!echo '#Surf*SurfTen' | gmx energy -f ener.edr -s system.tpr -b 500 -o surf-tens-x2.xvg

I got 42.7 dyn$\times$cm, that is roughly a 25% decrease with respect to the biphasic system. This number may change from run to run, depending for example on the distribution of pentanol between the two interfaces.

**TODO - explain what happens as you keep increasing the pentanol concentration.**

## Food for thought

### Effect of missing long-range dispersion forces

_Try to compute the surface tension of water, hexane or pentanol with its vapour. Note that in this case you need to run the production simulations at constant volume instead of constant pressure. Do the values still agree well with experients? Probably not. You may reflect on the reason of this disagrement (hint: it has to do with long-range dispersion forces and how they are commonly computed in MD simulations)._

### Finite size effects

...

### Surface free energy of viscoelastic materials

...